# Домашнее задание №3: Пляшущие человечки

Выполнил: Антон Бер <br>
Группа: MADE-ML-22

In [48]:
import string
from collections import Counter
import re
import random 
from copy import copy


import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams
from tqdm.notebook import tqdm

np.random.seed(23)

## Задание 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:
   - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
   - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
   - расшифруйте их таким частотным методом.



In [2]:
tokenizer = RegexpTokenizer(r"\w+")
alphabet_ru = " абвгдежзийклмнопрстуфхцчшщъыьэюя"
alphabet_eng = " " + string.ascii_lowercase

In [3]:
def tokenize(text, tokenizer=tokenizer, alphabet=alphabet_ru):
    text = "".join([char for char in text if char.lower() in alphabet])
    
    return " ".join(tokenizer.tokenize(text.lower()))

In [4]:
def get_frequencies(text, min_freq=0, n_gram=1):
    frequencies = dict()
    if n_gram > 1:
        text = ["".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        if value / len(text) > min_freq:
            frequencies[key] = value / len(text)
    
    return frequencies

In [5]:
def generate_random_mapping(frequencies):
    chars = list(frequencies.keys())
    replace_chars = np.random.choice(chars, replace=False, size=len(frequencies))
    mapping = dict()
    for original_char, replace_char in zip(chars, replace_chars):
        mapping[original_char] = replace_char
    
    return mapping

In [6]:
def apply_mapping(text, mapping):

    return "".join([mapping.get(char, "ь") for char in text])


In [7]:
def generate_reverse_mapping(corpus_frequencies, text_frequencies):
    corpus_frequencies_sorted = sorted(corpus_frequencies.items(), key=lambda x: x[1], reverse=True)
    text_frequencies_sorted = sorted(text_frequencies.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for text_char, text_freq in text_frequencies_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_frequencies_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff
        
        reverse_mapping[text_char] = best_char
        corpus_freqeuncies_sorted = [(char, freq) for char, freq in corpus_frequencies_sorted if char != best_char]
        
    return reverse_mapping

In [8]:
# score function
def char_accuracy(text_1, text_2):
    assert len(text_1) == len(text_2)
    right_chars = 0
    for c1, c2 in zip(text_1, text_2):
        right_chars += int(c1 == c2)
        
    return right_chars / len(text_1)

In [9]:
# score function
def levenstein_distance(text_1, text_2):
    assert len(text_1) == len(text_2)
    distances = range(len(text_1) + 1)
    for idx_2, char_2 in enumerate(text_2):
        new_dist = [idx_2 + 1]
        for idx_1, char_1 in enumerate(text_1):
            if char_1 == char_2:
                new_dist.append(distances[idx_1])
            else:
                new_dist.append(1 + min(distances[idx_1], distances[idx_1 + 1], new_dist[-1]))
        distances = new_dist

    return distances[-1]

In [10]:
# read data

with open("./data/AnnaKarenina.txt", "r", encoding="utf8") as file:
    corpus_1 = file.readlines()
    
with open("./data/WarAndPeace.txt", "r", encoding="utf8") as file:
    corpus_2 = file.readlines()
    
corpus = corpus_1 + corpus_2

tokenized_corpus = tokenize(" ".join(corpus))
corpus_frequencies = get_frequencies(tokenized_corpus)
mapping = generate_random_mapping(corpus_frequencies)

In [11]:
with open("./data/task_1_test_ru.txt", "r", encoding="utf8") as file:
    text = file.readlines()
    
tokenized_test = tokenize(" ".join(text))
encoded_test = apply_mapping(tokenized_test, mapping)
test_frequencies = get_frequencies(encoded_test)
tokenized_test

'на следующий день государь остановился в вишау лейбмедик вилье несколько раз был призываем к нему в главной квартире и в ближайших войсках распространилось известие что государь был нездоров он ничего не ел и дурно спал эту ночь как говорили приближенные причина этого нездоровья заключалась в сильном впечатлении произведенном на чувствительную душу государя видом раненых и убитых на заре го числа в вишау был препровожден с аванпостов французский офицер приехавший под парламентерским флагом требуя свидания с русским императором офицер этот был савари государь только что заснул и потому савари должен был дожидаться в полдень он был допущен к государю и через час поехал вместе с князем долгоруковым на аванпосты французской армии как слышно было цель присылки савари состояла в предложении свидания императора александра с наполеоном в личном свидании к радости и гордости всей армии было отказано и вместо государя князь долгоруков победитель при вишау был отправлен вместе с савари для перег

In [12]:
encoded_test

'цгтсеркхзиэщткрцмт нсхкгпмтнсягцнаэесжтатаэугхтерщдюркэштаэемртцрсшнемшнтпгфтдветопэфвагрютштцрюхтат егацнщтшагпяэпртэтатдеэыгщуэлтанщсшглтпгсопнсяпгцэенсмтэфарсяэртьянт нсхкгпмтдветцрфкнпнатнцтцэьр нтцртретэткхпцнтсогетбяхтцньмтшгшт нанпэеэтопэдеэырццвртопэьэцгтбян нтцрфкнпнамжтфгшезьгегсмтатсэемцнютаорьгяерцээтопнэфаркрццнютцгтьхасяаэяремцхзткхухт нсхкгпжтаэкнютпгцрцвлтэтхдэявлтцгтфгпрт нтьэсегтатаэугхтдветопропнаныкрцтстгагцонсянатъпгцйхфсшэщтнъэйрптопэрлгауэщтонктогпегюрцярпсшэютъег нютяпрдхжтсаэкгцэжтстпхссшэютэюорпгянпнютнъэйрптбянятдветсгагпэт нсхкгпмтянемшнтьянтфгсцхетэтонянюхтсгагпэткнеырцтдветкныэкгямсжтатонекрцмтнцтдветкнохирцтшт нсхкгпзтэтьрпрфтьгстонрлгетаюрсяртстшцжфрюткне нпхшнавютцгтгагцонсявтъпгцйхфсшнщтгпюээтшгштсевуцнтдвентйремтопэсвешэтсгагпэтснсянжегтатопркенырцээтсаэкгцэжтэюорпгянпгтгершсгцкпгтстцгонернцнютатеэьцнютсаэкгцээтштпгкнсяэтэт нпкнсяэтасрщтгпюээтдвентняшгфгцнтэтаюрсянт нсхкгпжтшцжфмткне нпхшнатондркэяремтопэтаэугхтдветняопгаерцтаюрсяртстсгагпэткежторпр 

In [13]:
reverse_mapping = generate_reverse_mapping(corpus_frequencies, test_frequencies)
decoded_test = apply_mapping(encoded_test, reverse_mapping)
decoded_test

'та ттаркюфай рать яоткраиь откатосаттч с сахак тайчуарау сатьа татуотьуо иач чьт киачьсаау у таук с ятастой усаикаиа а с чтахайхаю сойтуаю иаткиоткиататоть ачсаткаа чко яоткраиь чьт тачроиос от тачаяо та ат а ркито ткат экк точь уау яосоиата киачтахаттьа киачата экояо тачроиосьч чаутючатать с татьтоу скачактатаа киоачсараттоу та чкстксакатьткю ркхк яоткраич сароу иататью а кчакью та чаиа яо чатта с сахак чьт киакиосохрат т асаткоткос юиатюкчтуай оюаюаи киааюасхай кор каитауаткаитуау ютаяоу киачкч тсаратач т икттуау аукаиакоиоу оюаюаи экок чьт тасаиа яоткраиь котьуо чко чатткт а кокоук тасаиа ротхат чьт рохаракьтч с котрать от чьт роккфат у яоткраию а чаиач чат коаюат суатка т утччау ротяоикуосьу та асаткоткь юиатюкчтуой аиуаа уау ттьхто чьто юать киатьтуа тасаиа тоткочта с киартохатаа тсаратач аукаиакоиа атаутатриа т такотаотоу с тачтоу тсаратаа у иаротка а яоиротка стай аиуаа чьто окуачато а суатко яоткраич утччь ротяоикуос кочаракать киа сахак чьт оккиастат суатка т тасаиа ртч каиая

In [14]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 34.64%
Levenstein distace: 767


Частостный метод без модификаций не позволяет качественно декодировать текст.

А что по поводу английских текстов?

In [15]:
with open("./data/WarAndPeaceEng.txt", "r", encoding="utf8") as file:
    eng_text = file.readlines()

tokenized_corpus_eng = tokenize(" ".join(eng_text), tokenizer=tokenizer, alphabet=alphabet_eng)
corpus_frequencies_eng = get_frequencies(tokenized_corpus_eng)
mapping_eng = generate_random_mapping(corpus_frequencies_eng)

In [16]:
with open("./data/task_1_test_eng.txt", "r", encoding="utf8") as file:
    text = file.readlines()
    
tokenized_test = tokenize(" ".join(text), tokenizer=tokenizer, alphabet=alphabet_eng)
encoded_test = apply_mapping(tokenized_test, mapping_eng)
test_frequencies_eng = get_frequencies(encoded_test)
tokenized_test

'happy as petya was he felt sad at having to go home knowing that all the enjoyment of that day was over he did not go straight home from the kremlin but called on his friend obolenski who was fifteen and was also entering the regiment on returning home petya announced resolutely and firmly that if he was not allowed to enter the service he would run away and next day count ilya rostovthough he had not yet quite yieldedwent to inquire how he could arrange for petya to serve where there would be least danger'

In [17]:
encoded_test

'xkss ckucsnh kcvkucxnconbhcuktckhcxklerqchgcqgcxgzncfrgverqchxkhckbbchxncnrpg znrhcgochxkhctk cvkucglnycxnctetcrghcqgcuhykeqxhcxgzncoygzchxncfynzbercdahcikbbntcgrcxeucoyenrtcgdgbnrufecvxgcvkucoeohnnrckrtcvkuckbugcnrhnyerqchxncynqeznrhcgrcynhayrerqcxgzncsnh kckrrgarintcynugbahnb ckrtcoeyzb chxkhceocxncvkucrghckbbgvntchgcnrhnychxncunyleincxncvgabtcyarckvk ckrtcrnjhctk cigarhceb kcyguhglhxgaqxcxncxktcrghc nhcwaehnc enbtntvnrhchgcerwaeyncxgvcxncigabtckyykrqncogycsnh kchgcunylncvxnynchxnyncvgabtcdncbnkuhctkrqny'

In [18]:
reverse_mapping = generate_reverse_mapping(corpus_frequencies_eng, test_frequencies_eng)
decoded_test = apply_mapping(encoded_test, reverse_mapping)
decoded_test

'savvu al vetua lal se fedt lad at savdau ta ua saye kaaldau tsat add tse eaxauyeat af tsat dau lal aver se ddd aat ua ltradust saye fray tse kreydda kut vadded aa sdl frdead akadealkd lsa lal fdfteea aad lal adla eaterdau tse reudyeat aa returadau saye vetua aaaauaved reladutedu aad fdrydu tsat df se lal aat addaled ta eater tse lervdve se laudd rua alau aad aext dau vauat ddua raltavtsauus se sad aat uet xudte udeddedleat ta daxudre sal se vaudd arraaue far vetua ta lerve lsere tsere laudd ke dealt daauer'

In [19]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 58.51%
Levenstein distace: 212


В английском языке меньше букв, чем в русском и, возможно, по этой причине качество выше, т.к. проще "попасть" по частоте в правильные варианты проще.

## Задание 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм


In [27]:
# read data

with open("./data/AnnaKarenina.txt", "r", encoding="utf8") as file:
    corpus_1 = file.readlines()
    
with open("./data/WarAndPeace.txt", "r", encoding="utf8") as file:
    corpus_2 = file.readlines()
    
corpus = corpus_1 + corpus_2

tokenized_corpus = tokenize(" ".join(corpus))
corpus_frequencies = get_frequencies(tokenized_corpus)
mapping = generate_random_mapping(corpus_frequencies)

In [25]:
corpus_frequencies_bigram = get_frequencies(tokenized_corpus, n_gram=2)
test_frequencies_bigram = get_frequencies(encoded_test, n_gram=2)

corpus_frequencies_sorted = sorted(corpus_frequencies_bigram.items(), key=lambda x: x[1], reverse=True)
test_frequencies_sorted = sorted(test_frequencies_bigram.items(), key=lambda x: x[1], reverse=True)

In [26]:
def generate_reverse_mapping_bigram(corpus_frequencies, text_frequencies):
    corpus_frequencies_sorted = sorted(corpus_frequencies.items(), key=lambda x: x[1], reverse=True)
    text_frequencies_sorted = sorted(text_frequencies.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    
    for i, (text_bigram, text_freq) in enumerate(text_frequencies_sorted):
        filtered_frequencies = copy(corpus_frequencies_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtered_frequencies = [(bigram, freq) for bigram, freq in filtered_frequencies 
                                    if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtered_frequencies = [(bigram, freq) for bigram, freq in filtered_frequencies 
                                    if bigram[1] == reverse_mapping[text_bigram[1]]]
            
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtered_frequencies:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
        
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
        
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1] 
    
    return reverse_mapping

In [28]:
with open("./data/task_1_test_ru.txt", "r", encoding="utf8") as file:
    text = file.readlines()
    
tokenized_test = tokenize(" ".join(text))
encoded_test = apply_mapping(tokenized_test, mapping)
test_frequencies = get_frequencies(encoded_test)
tokenized_test

'на следующий день государь остановился в вишау лейбмедик вилье несколько раз был призываем к нему в главной квартире и в ближайших войсках распространилось известие что государь был нездоров он ничего не ел и дурно спал эту ночь как говорили приближенные причина этого нездоровья заключалась в сильном впечатлении произведенном на чувствительную душу государя видом раненых и убитых на заре го числа в вишау был препровожден с аванпостов французский офицер приехавший под парламентерским флагом требуя свидания с русским императором офицер этот был савари государь только что заснул и потому савари должен был дожидаться в полдень он был допущен к государю и через час поехал вместе с князем долгоруковым на аванпосты французской армии как слышно было цель присылки савари состояла в предложении свидания императора александра с наполеоном в личном свидании к радости и гордости всей армии было отказано и вместо государя князь долгоруков победитель при вишау был отправлен вместе с савари для перег

In [29]:
reverse_mapping_bigram = generate_reverse_mapping_bigram(corpus_frequencies_bigram, test_frequencies_bigram)
decoded_test = apply_mapping(encoded_test, reverse_mapping_bigram)
decoded_test

'насюлы чбьисс ын сютюч аь стюьанттилюястстиначслыстэы иостил ысныюотл отсьаистклсюьииктаыэсосныэчстсюлатнтссотаььиьысистстлиеасниосттсюоаосьаююьтюььанилтю сиитыюьиыс ьтсютюч аь стклсныи тьттстнсни ыютснысылсис чьнтсююалссьчснт  соаосютттьилисюьитлиеыннкысюьи инассьтютсныи тьтт ясиаолб алаю стсюил нтэстюы аьлыниисюьтииты ыннтэснас чтюьтиьыл нчбс чнчсютюч аьясти тэсьанынкосисчтиькоснасиаьысютс июластстиначстклсюьыюьттте ынсюсатанютюьттсяьанлчиюоисстяилыьсюьиыоатниссют сюаьлаэыньыьюоиэсялаютэсььытчясюти аниясюсьчююоиэсиэюыьаьтьтэстяилыьссьтьстклсюатаьисютюч аь сьтл отс ьтсиаюнчлсисютьтэчсюатаьис тлеынстклс теи аь юястсютл ын стнстклс тючьынсосютюч аьбсис ыьыис аюсютыоалстэыюьысюсоняиыэс тлютьчотткэснасатанютюьксяьанлчиюотссаьэиисоаосюлкннтстклтслыл сюьиюклоисюатаьисютюьтяластсюьы лтеыниисюти аниясиэюыьаьтьасалыоюан ьасюснаютлытнтэстсли нтэсюти аниисосьа тюьисисють тюьистюыссаьэиистклтстьоаиантсистэыюьтсютюч аьясоняи с тлютьчоттсютты иьыл сюьистиначстклстьюьатлынстэыюьысюсюатаьис лясюыьыю

In [30]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 24.94%
Levenstein distace: 869


Метод частотных биграмм работает еще хуже, чем посимвольный частотный по тем же причинам: вариантов гораздо больше.

## Задание 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


**Метод:**<br>
Биграммы в тексте - звенья марковской цепи, а частота биграмм - вероятность перехода.
Для обучения считается вероятность порождение текста как произведение всех биграмм, входящих в него.


In [31]:
def get_frequencies(text, min_freq=0, n_gram=2):
    frequencies = dict()
    length = len(set(text)) ** n_gram # all possible n_grams
    
    text = ["".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        frequencies[key] = (value + 1) / (len(text) + length) # to avoid absent n_grams
        
    return frequencies

In [32]:
def get_text_probability(text, mapping, frequencies, n_gram=2, alphabet=alphabet_ru):
    decoded_text = apply_mapping(text, mapping)
    
    log_probability = 0
    for i in range(len(decoded_text) - n_gram):
        bigram = decoded_text[i: i + n_gram]
        absent_bigram_probability = 1 / (len(text) + len(alphabet) ** n_gram) # to avoid 0 probs
        bigram_probability = frequencies.get(bigram, absent_bigram_probability)
        
        log_probability += np.log(bigram_probability)
    
    return log_probability

In [187]:
def generate_mcmc_reverse_mapping(encoded_text, alphabet_encoded, alphabet_corpus, corpus_frequencies, 
                                  n_iters=10000, epochs=10, n_gram=2, verbose=True):
    
    accepted = 0
    best_mapping = None
    best_ll = -np.inf # log-likelihood for scoring
    
    for epoch in range(epochs):
        
        alphabet_encoded = list(alphabet_encoded)
        alphabet_iter = list(alphabet_corpus)
        reverse_mapping = {key: value for key, value 
                               in zip(alphabet_encoded, alphabet_iter[:len(alphabet_encoded)])}
        current_log_probability = get_text_probability(encoded_text, reverse_mapping, corpus_frequencies, n_gram=n_gram)
        
        for i in tqdm(range(n_iters), total=n_iters, leave=False, disable=not verbose):
            alphabet_proposal = alphabet_iter[:]
            idx_1, idx_2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[idx_1], alphabet_proposal[idx_2] = alphabet_proposal[idx_2], alphabet_proposal[idx_1]
            reverse_mapping_proposal = {key: value for key, value 
                               in zip(alphabet_encoded, alphabet_proposal[:len(alphabet_encoded)])}
            proposal_log_probability = get_text_probability(encoded_text, reverse_mapping_proposal, corpus_frequencies, n_gram=n_gram)
            
            p_accept = np.exp(proposal_log_probability - current_log_probability)
            
            if p_accept > np.random.rand():
                accepted += 1
                alphabet_iter = alphabet_proposal
                current_log_probability = proposal_log_probability
                reverse_mapping = reverse_mapping_proposal
                
        if current_log_probability > best_ll:
            best_ll = current_log_probability
            best_mapping = reverse_mapping
        print(f"Epoch #{epoch + 1}")
        print(f"Current log-likelihood: {current_log_probability}")
        print(f"Current accepted proposals: {accepted}")
 
    print(f"Best likelihood: {best_ll}")
    print(f"Acceptance ratio: {accepted / (n_iters * epochs)}")

    return best_mapping

In [56]:
corpus_frequencies = get_frequencies(tokenized_corpus, n_gram=2)

In [64]:
EPOCHS = 5

In [65]:
reverse_mapping = generate_mcmc_reverse_mapping(encoded_test, alphabet_ru, alphabet_ru, corpus_frequencies, epochs=EPOCHS)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #1
Current log-likelihood: -6725.3750723424455
Current accepted proposals: 97


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #2
Current log-likelihood: -6573.68144094178
Current accepted proposals: 239


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #3
Current log-likelihood: -7377.9340985962035
Current accepted proposals: 340


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #4
Current log-likelihood: -6573.68144094178
Current accepted proposals: 460


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #5
Current log-likelihood: -6573.68144094178
Current accepted proposals: 582
Best likelihood: -6573.68144094178
Acceptance ratio: 0.01164


In [66]:
decoded_test = apply_mapping(encoded_test, reverse_mapping)

In [69]:
decoded_test

'на следующий день государь остановился в вишау лейбмедик вилье несколько раз был призываем к нему в главной квартире и в ближайших войсках распространилось известие что государь был нездоров он ничего не ел и дурно спал эту ночь как говорили приближенные причина этого нездоровья заключалась в сильном впечатлении произведенном на чувствительную душу государя видом раненых и убитых на заре го числа в вишау был препровожден с аванпостов французский офицер приехавший под парламентерским флагом требуя свидания с русским императором офицер этот был савари государь только что заснул и потому савари должен был дожидаться в полдень он был допущен к государю и через час поехал вместе с князем долгоруковым на аванпосты французской армии как слышно было цель присылки савари состояла в предложении свидания императора александра с наполеоном в личном свидании к радости и гордости всей армии было отказано и вместо государя князь долгоруков победитель при вишау был отправлен вместе с савари для перег

In [67]:
print(f"Char decoding accuracy: {char_accuracy(tokenized_test, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(tokenized_test, decoded_test)}")

Char decoding accuracy: 100.00%
Levenstein distace: 0


Текст расшифрован, метрики идеальные.

## Задание 4

Расшифруйте сообщение:

In [134]:
message_1 = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
message_2 = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [135]:
corpus_frequencies = get_frequencies(tokenized_corpus, n_gram=2)
message_frequencies = get_frequencies(message_1, n_gram=2)

In [136]:
corpus_frequencies_sorted = sorted(corpus_frequencies.items(), key=lambda x: x[1], reverse=True)
message_frequencies_sorted = sorted(message_frequencies.items(), key=lambda x: x[1], reverse=True)

In [146]:
alphabet_message = " " + "".join([char for char in set(message_1)])

In [148]:
EPOCHS = 50

In [149]:
reverse_mapping = generate_mcmc_reverse_mapping(message_1, alphabet_message, alphabet_ru, corpus_frequencies, epochs=EPOCHS)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #1
Current log-likelihood: -1361.3459252159712
Current accepted proposals: 546


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #2
Current log-likelihood: -1267.0379756685018
Current accepted proposals: 1112


  0%|          | 0/10000 [00:00<?, ?it/s]

...

Epoch #48
Current log-likelihood: -1237.4064191767038
Current accepted proposals: 23284


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #49
Current log-likelihood: -1360.9351828851065
Current accepted proposals: 23718


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #50
Current log-likelihood: -1265.3307828296981
Current accepted proposals: 24210
Best likelihood: -1231.2547551943435
Acceptance ratio: 0.04842


In [150]:
apply_mapping(message_1, reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаф'

In [151]:
message_answer = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
decoded_test = apply_mapping(message_1, reverse_mapping)

In [152]:
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")

Char decoding accuracy: 93.48%
Levenstein distace: 15


Сообщение расшифровано почти верно, метод работает!


Проверим для второго сообщения.

In [154]:
alphabet_message = " " + "".join([char for char in set(message_2)])

In [160]:
reverse_mapping = generate_mcmc_reverse_mapping(message_2, alphabet_message, alphabet_ru, corpus_frequencies, epochs=50)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #1
Current log-likelihood: -1345.8755584771795
Current accepted proposals: 490


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #2
Current log-likelihood: -1349.6720447515313
Current accepted proposals: 945


  0%|          | 0/10000 [00:00<?, ?it/s]

...

Epoch #48
Current log-likelihood: -1371.9986912290806
Current accepted proposals: 22250


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #49
Current log-likelihood: -1368.827399342378
Current accepted proposals: 22688


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #50
Current log-likelihood: -1234.80586248601
Current accepted proposals: 23142
Best likelihood: -1234.80586248601
Acceptance ratio: 0.046284


In [161]:
decoded_test = apply_mapping(message_2, reverse_mapping)
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")
decoded_test

Char decoding accuracy: 94.78%
Levenstein distace: 12


'если вы вимите нордальный или почти нордальный текст у этого соожщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса хотя конечно я ничего не ожещаш'

Работает и для второго сообщения.

## Бонус: Задание 5

 а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [164]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
alphabet_message = " " + "".join([char for char in set(message)])

#### Триграммы

In [163]:
corpus_frequencies_trigramm = get_frequencies(tokenized_corpus, n_gram=3)

In [165]:
reverse_mapping = generate_mcmc_reverse_mapping(message, alphabet_message, alphabet_ru, corpus_frequencies_trigramm, n_gram=3, epochs=25)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #1
Current log-likelihood: -2111.264450847848
Current accepted proposals: 639


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #2
Current log-likelihood: -1731.7789239524675
Current accepted proposals: 1181


  0%|          | 0/10000 [00:00<?, ?it/s]

...

Epoch #23
Current log-likelihood: -1988.0324017309592
Current accepted proposals: 13290


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #24
Current log-likelihood: -1766.825080063892
Current accepted proposals: 13908


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #25
Current log-likelihood: -2083.6312083829284
Current accepted proposals: 14488
Best likelihood: -1725.6602016838426
Acceptance ratio: 0.057952


In [166]:
decoded_test = apply_mapping(message, reverse_mapping)
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")
decoded_test

Char decoding accuracy: 99.57%
Levenstein distace: 1


'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаж'

Триграммы сработали лучше, чем биграммы

#### 4-граммы


In [169]:
corpus_frequencies_fourgramm = get_frequencies(tokenized_corpus, n_gram=4)
reverse_mapping = generate_mcmc_reverse_mapping(message, alphabet_message, alphabet_ru, corpus_frequencies_fourgramm, n_gram=4, epochs=25)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #1
Current log-likelihood: -2819.688813112943
Current accepted proposals: 650


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #2
Current log-likelihood: -2184.5921398183245
Current accepted proposals: 1137


  0%|          | 0/10000 [00:00<?, ?it/s]

...

Epoch #23
Current log-likelihood: -2184.5921398183245
Current accepted proposals: 14513


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #24
Current log-likelihood: -2184.5921398183245
Current accepted proposals: 15009


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch #25
Current log-likelihood: -2691.3299750618735
Current accepted proposals: 15665
Best likelihood: -2184.5921398183245
Acceptance ratio: 0.06266


In [170]:
decoded_test = apply_mapping(message, reverse_mapping)
print(f"Char decoding accuracy: {char_accuracy(message_answer, decoded_test) * 100:.2f}%")
print(f"Levenstein distace: {levenstein_distance(message_answer, decoded_test)}")
decoded_test

Char decoding accuracy: 99.57%
Levenstein distace: 1


'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаф'

Триграммы и 4-граммы работают примерно одинаково

## Бонус: Задание 6

Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Потенциальные способы использования модели:
1. Дешифровка записей, закодированных различными методами перестановок.
2. Анализ мутаций в аминокислотных последовательностях.